In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

print('hello world')

In [ ]:
# Load the S&P 500 data
df_sp500 = pd.read_csv('data/sp500_2015-2025.csv')

# Convert the 'Date' column to datetime format for better plotting
df_sp500['Date'] = pd.to_datetime(df_sp500['Date'])

# Sort the dataframe by date (ascending order)
df_sp500 = df_sp500.sort_values('Date')

# Conver string to float
df_sp500['Close/Last'] = df_sp500['Close/Last'].astype(float)
df_sp500['Open'] = df_sp500['Open'].astype(float)
df_sp500['High'] = df_sp500['High'].astype(float)
df_sp500['Low'] = df_sp500['Low'].astype(float)

# Display the first few rows of the data
df_sp500.head()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(12, 6))

# Plot a single line that shows closing prices
plt.plot(df_sp500['Date'], df_sp500['Close/Last'], label='Close Price', color='blue', alpha=0.7)

# Customize the plot
plt.title('S&P 500 Close/Last Value')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Percentage change of close/last value

# Calculate the percentage change from the first day
first_price = df_sp500['Close/Last'].iloc[0]
df_sp500['Percent_Change'] = ((df_sp500['Close/Last'] - first_price) / first_price) * 100

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(df_sp500['Date'], df_sp500['Percent_Change'])
plt.title('S&P 500 Percentage Change (Starting at 0%)')
plt.xlabel('Date')
plt.ylabel('Percentage Change (%)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Simulate DCA strategy

# Filter for only Fridays (weekday 4 is Friday in Python)
fridays = df_sp500[df_sp500['Date'].dt.weekday == 4]

# Initialize variables for DCA strategy
initial_investment = 5000
weekly_investment = 100
total_invested = initial_investment
shares_owned = initial_investment / fridays.iloc[0]['Close/Last']
investment_value = []
investment_cost = []
dates = []

# Start with initial investment
dates.append(fridays.iloc[0]['Date'])
investment_value.append(shares_owned * fridays.iloc[0]['Close/Last'])
investment_cost.append(total_invested)

# Simulate buying $100 worth of shares every Friday
for i in range(1, len(fridays)):
    # Buy more shares with weekly investment
    new_shares = weekly_investment / fridays.iloc[i]['Close/Last']
    shares_owned += new_shares
    total_invested += weekly_investment
    
    # Record data
    dates.append(fridays.iloc[i]['Date'])
    investment_value.append(shares_owned * fridays.iloc[i]['Close/Last'])
    investment_cost.append(total_invested)

# Create a dataframe to hold our results
dca_results = pd.DataFrame({
    'Date': dates,
    'Total Invested': investment_cost,
    'Portfolio Value': investment_value,
    'Profit/Loss': np.array(investment_value) - np.array(investment_cost),
    'Return %': ((np.array(investment_value) - np.array(investment_cost)) / np.array(investment_cost)) * 100
})

# Plot the results
plt.figure(figsize=(14, 8))

# Plot total invested vs portfolio value
plt.plot(dca_results['Date'], dca_results['Total Invested'], label='Total Invested ($)', color='blue')
plt.plot(dca_results['Date'], dca_results['Portfolio Value'], label='Portfolio Value ($)', color='green')

# Highlight profit/loss region
plt.fill_between(dca_results['Date'], dca_results['Total Invested'], dca_results['Portfolio Value'], 
                 where=(dca_results['Portfolio Value'] >= dca_results['Total Invested']),
                 color='green', alpha=0.3, label='Profit Region')
plt.fill_between(dca_results['Date'], dca_results['Total Invested'], dca_results['Portfolio Value'], 
                 where=(dca_results['Portfolio Value'] <= dca_results['Total Invested']),
                 color='red', alpha=0.3, label='Loss Region')

# Customize the plot
plt.title('Dollar Cost Averaging: (5,000 + 100x) Investment', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Value ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

# Display summary statistics
print("DCA Strategy Summary:")
print(f"Start Date: {dca_results['Date'].min().strftime('%Y-%m-%d')}")
print(f"End Date: {dca_results['Date'].max().strftime('%Y-%m-%d')}")
print(f"Total Amount Invested: ${dca_results['Total Invested'].iloc[-1]:.2f}")
print(f"Final Portfolio Value: ${dca_results['Portfolio Value'].iloc[-1]:.2f}")
profit_loss = dca_results['Portfolio Value'].iloc[-1] - dca_results['Total Invested'].iloc[-1]
print(f"Total Profit/Loss: ${profit_loss:.2f} ({profit_loss/dca_results['Total Invested'].iloc[-1]*100:.2f}%)")
print(f"Total Number of Shares Owned: {shares_owned:.2f}")
print(f"Average Cost Per Share: ${dca_results['Total Invested'].iloc[-1]/shares_owned:.2f}")
print(f"Final Share Price: ${fridays.iloc[-1]['Close/Last']:.2f}")

In [ ]:
# Create a percentage comparison between S&P 500 and DCA strategy
plt.figure(figsize=(14, 8))

# Calculate percentage return for S&P 500 (only on Friday dates to match DCA strategy)
sp500_fridays = df_sp500[df_sp500['Date'].isin(dca_results['Date'])]
first_sp500_price = sp500_fridays.iloc[0]['Close/Last']
sp500_percent_change = ((sp500_fridays['Close/Last'] - first_sp500_price) / first_sp500_price) * 100

# Plot S&P 500 percentage change
plt.plot(dca_results['Date'], sp500_percent_change.values, 
         label='S&P 500 Buy & Hold', color='blue', linewidth=2)

# Plot DCA strategy percentage return
plt.plot(dca_results['Date'], dca_results['Return %'], 
         label='DCA Strategy', color='green', linewidth=2)

# Highlight the difference between strategies
plt.fill_between(dca_results['Date'], sp500_percent_change.values, dca_results['Return %'],
                where=(dca_results['Return %'] >= sp500_percent_change.values),
                color='green', alpha=0.3, label='DCA Outperforms')
plt.fill_between(dca_results['Date'], sp500_percent_change.values, dca_results['Return %'],
                where=(dca_results['Return %'] <= sp500_percent_change.values),
                color='red', alpha=0.3, label='S&P 500 Outperforms')

# Customize the plot
plt.title('Percentage Return: S&P 500 vs. DCA Strategy', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Return (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='best', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

# Display comparison statistics
print("Performance Comparison (Start to End):")
final_sp500_return = sp500_percent_change.iloc[-1]
final_dca_return = dca_results['Return %'].iloc[-1]
print(f"S&P 500 Buy & Hold Return: {final_sp500_return:.2f}%")
print(f"DCA Strategy Return: {final_dca_return:.2f}%")
print(f"Difference: {final_dca_return - final_sp500_return:.2f}%")

# Calculate and display annualized returns
days_passed = (dca_results['Date'].max() - dca_results['Date'].min()).days
years = days_passed / 365
sp500_annual_return = ((1 + final_sp500_return/100) ** (1/years) - 1) * 100
dca_annual_return = ((1 + final_dca_return/100) ** (1/years) - 1) * 100
print(f"\nAnnualized Returns:")
print(f"S&P 500 Buy & Hold: {sp500_annual_return:.2f}%")
print(f"DCA Strategy: {dca_annual_return:.2f}%")